### Requirements/Initial Setup

1. Create an Application in AAD - https://portal.azure.com/#view/Microsoft_AAD_IAM/ActiveDirectoryMenuBlade/~/Overview
2. Note the `ClientId`, `Tenant/DirectoryId`, `SecretValue`, `SecretId`
3. Go to the Storage Account -> IAM role and assign the role  `Storage Blob Data Contributor` to the application (via the option `Assign access to User, group or service principal`)
4. Create a new Azure Key Vault (for detailed screenshots refer to https://hevodata.com/learn/databricks-secret/)
5. Create a Secret and enter the `SecretValue` from (2) above and note down the Secret Name (e.g. `BlobStoreAccessKey`)
6. Go to the Databricks URL e.g. https://adb-<some_id>.azuredatabricks.net/?o=<some_id> and add `#secrets/createScope` to the URL to create a Databricks scope. (e.g. `azure_delta_lake_access_scope`)

Add the following spark configs (pyspark) to allow the access to the storage account. (for additional options refer to - https://docs.databricks.com/storage/azure-storage.html)

```
service_credential = dbutils.secrets.get(scope="<scope>",key="<service-credential-key>")

spark.conf.set("fs.azure.account.auth.type.<storage-account>.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.<storage-account>.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.<storage-account>.dfs.core.windows.net", "<application-id>")
spark.conf.set("fs.azure.account.oauth2.client.secret.<storage-account>.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.<storage-account>.dfs.core.windows.net", "https://login.microsoftonline.com/<directory-id>/oauth2/token")
```

In [ ]:

dbutils.fs.ls("abfss://CONTAINER@STORAGE_ACCOUNT.dfs.core.windows.net/<path>")

In [ ]:
%sql
SELECT patient_city, gender, COUNT(count) AS `Number of Patients`
    FROM delta.`abfss://CONTAINER@STORAGE_ACCOUNT.dfs.core.windows.net/delta_lake_poc/gold`
    GROUP BY patient_city, gender

### Register External tables with Databricks
To register the able in Databricks catalog (can be either Unity or Hive metastore) the table should be created as an EXTERNAL table (aka Unmanaged table). With this registration, only the metadata is registered with Databricks and the data is still in the ADLSg2 folders. With this even if the table is dropped in the Databricks catalog the data is not deleted. See - https://docs.databricks.com/lakehouse/data-objects.html#what-is-an-unmanaged-table for further details.

1. Create metastore - To register the unmanaged table a Metastore and Catalog needs to be created first. For example you can go to the `Manage Account` option and click on the `Data` icon/option to create a new Unity metastore. (say this mwetastore name is - `main`). In order to create a metastore, you would need to go to Azure portal and first create an `Access Connector for Azure Databricks`. When creating a metastore in Unity you would need to enter the ADLSg2 folder path as well as the Access Connector Id (resourceId for the Access Connector that was just created in Azure). Note that currently Unity catalog only allows 1 metastore per region, so please define the ADLSg2 path accordingly.
2. Assign one or more Databricks workspaces that neeed to use the metastore to the created metastore.
3. The Databricks cluster also would need to be assigned to work with Unity catalog. This can be done by just creating a new cluster.
4. Register an External location to point to the ADLSg2 folder that needs to be accessed via the metastore.  
5. Once then follow the steps below to create the table.

#### Create an External Location in Unity Catalog
Before you can create an external table, you must create a storage credential that allows Unity Catalog to read from and write to the path on your cloud tenant, and an external location that references it.
##### Create a storage credential
Follow these steps to create a storage credential using the Data Explorer.

1. In the sidebar, set the persona to SQL.
2. Click Data.
3. Click Storage Credentials.
4. Click Create Credential.
5. Enter `azureblobstorekey` for he name of the storage credential.
6. Set `SecretValue`, `Tenant/DirectoryId`, and `ClientId` to the values for your service principal.
7 Optionally enter a comment for the storage credential.
8. Click Save.

##### Create an external location
An external location references a storage credential (created above) and points to an ADLSg2 folder. The external location allows reading from and writing to only that path and its child directories. Follow these steps to create an external location using the Data Explorer.

1. Click Data.
2. Click External Locations.
3. Click Create location.
4. Enter `deltatablepoc` for the name of the external location.
5. Enter the storage container path for the location allows reading from or writing to.
6. Set Storage Credential to example_credential to the storage credential you just created.
7. Optionally enter a comment for the external location.
8. Click Save.

In [ ]:
%sql

-- Create external table with Unity catalog metastore
CREATE TABLE IF NOT EXISTS main.default.patientGold USING DELTA LOCATION 'abfss://CONTAINER@STORAGE_ACCOUNT.dfs.core.windows.net/delta_lake_poc/gold';

SELECT patient_city, gender, COUNT(count) AS `Number of Patients`
    FROM main.default.patientGold
    GROUP BY patient_city, gender

In [ ]:
%sql

CREATE TABLE IF NOT EXISTS main.default.patientSilver USING DELTA LOCATION 'abfss://CONTAINER@STORAGE_ACCOUNT.dfs.core.windows.net/delta_lake_poc/silver';

SELECT * FROM main.default.patientSilver;

In [ ]:
%sql

-- Create external table on Hive metastore
-- NOTE with Hive metastore dropping that schema using the CASCADE option causes all files in that schema location to be deleted recursively, regardless of the table type (managed or external).

CREATE TABLE IF NOT EXISTS patientGold4 USING DELTA LOCATION 'abfss://CONTAINER@STORAGE_ACCOUNT.dfs.core.windows.net/delta_lake_poc/gold';

SELECT patient_city, gender, COUNT(count) AS `Number of Patients`
    FROM patientGold4
    GROUP BY patient_city, gender